# Panopic Point-Cloud Construction

## Detectron2 (Panopic Segmentation)

URL: https://github.com/facebookresearch/detectron2

**License**: Apache License, Version 2.0

**Prerequisite**:

- detectron2

- vid2depth_tf2

### Setup

In [ ]:
# Some basic setup:
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

# import some common libraries
import numpy as np
import pandas as pd
import os
import json
import cv2
import tqdm
from pyntcloud import PyntCloud

In [ ]:
img_width = 416
img_height = 128

### Make Directory

In [ ]:
!mkdir panopic_img_kitti
!mkdir panopic_edge_img_kitti
!mkdir check_img_kitti
!mkdir seg_depth_img_kitti
!mkdir chk_image_kitti
!mkdir seg_depth_img_kitti
!mkdir semantic_point_cloud_kitti
panopic_img_kitti = 'panopic_img_kitti'
panopic_edge_img_kitti = '
panopic_edge_img_kitti'
check_img_kitti = 'check_img_kitti'
seg_depth_img_kitti = 'seg_depth_img_kitti'
chk_image_kitti = "chk_image_kitti"
seg_depth_img_kitti = "seg_depth_img_kitti"

### File Read for Input Image

In [ ]:
path_depth = "./vid2depth_tf2/data/kitti_raw_eigen/2011_09_26_drive_0001_sync_02/"
files = os.listdir(path_depth)
files = sorted(files)

count = 0
for file_name in files:
    if '.jpg' in file_name:
        count +=1
print('Number of Files: {}'.format(count))

### Setting Configuration of Detectron2

In [ ]:
from detectron2.config import get_cfg
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml"))
cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-PanopticSegmentation/panoptic_fpn_R_101_3x.yaml")

### Input Video (Option)

In [ ]:
vidcap = cv2.VideoCapture('big_buck_bunny_720p_5mb.mp4')
success,image = vidcap.read()
count = 0
while success:
  cv2.imwrite("frame%d.jpg" % count, image)     # save frame as JPEG file      
  success,image = vidcap.read()
  print('Read a new frame: ', success)
  count += 1

### Inferencing Panopic Segmentation

In [ ]:
img_count = 0
for file_name in files:
    if '.jpg' in file_name:
        img_count += 1

        file_path = path_depth+file_name
        img = cv2.imread(file_path)
        img = np.asarray(img, dtype=np.float32)
        original_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        original_img = original_img[:128, :416]
        y, x, s = np.shape(original_img)
        
        predictor = DefaultPredictor(cfg)
        panoptic_seg, segments_info = predictor(original_img)["panoptic_seg"]
        v = Visualizer(original_img[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.2)
        
        panopic_seg_img = v.draw_panoptic_seg_predictions(panoptic_seg.to("cpu"), segments_info)
        #panopic_seg_img = np.asarray(panopic_seg_img.get_image()[:, :, ::-1])
        panopic_seg_img = np.asarray(panopic_seg_img.get_image())
        panopic_seg_img = cv2.resize(panopic_seg_img, (x, y))
        
        #print('{}: {}'.format(img_count, file_name))
        #plt.imshow(original_img/255.0)
        #plt.show()        
        #plt.imshow(panopic_seg_img)
        #plt.show()
        
        cv2.imwrite(panopic_img_kitti+"/"+file_name, panopic_seg_img)

files = os.listdir(panopic_img_kitti) 
count = len(files)
print('Number of Files: {}'.format(count))
print("Panopic Segmentation is done.")

## vid2depth (Depth Estimation)
### vid2depth_tf2 (TensorFlow 2.0 version)
URL: https://github.com/IAMAl/vid2depth_tf2

**NOTE**: use *inference.py* in this directory.

**License**: Apache License, Version 2.0

### Inferencing Depth Estimation

In [ ]:
!python3 ./vid2depth_tf2/inference.py \
  --output_dir   ./depth_img \
  --kitti_dir   ./vid2depth_tf2/ \
  --kitti_video data/kitti_raw_eigen/2011_09_26_drive_0001_sync  \
  --model_ckpt  ./vid2depth_tf2/checkpoints/model-119496 \
  --batch_size  5 \
  --img_width   img_width \
  --img_height  img_height


path_depth_inf = "./depth_img"
files_depth = os.listdir(path_depth_inf)
count = len(files_depth)
print('Number of Files: {}'.format(count))
print("Depth Estimation is done.")

## Edge Detection

In [ ]:
clr_th = 0.125
scl_th = 2
img_count = 0
for file_name in files:
    if '.jpg' in file_name:
        img_count += 1
        
        file_path = path_depth+file_name
        
        file_path = panopic_img_kitti+"/"+file_name
        sgmnt_img = cv2.imread(file_path)
        sgmnt_img = np.asarray(sgmnt_img, dtype=np.float32)
        sgmnt_img = cv2.cvtColor(sgmnt_img, cv2.COLOR_BGR2RGB)
        sgmnt_img = cv2.resize(sgmnt_img, (416, 128))
        
        edges_img = np.zeros((128, 416, 3))
        for y in range(scl_th, 128):
            for x in range(scl_th, 416):
                
                pxl = np.zeros((scl_th, scl_th, 3))
                for py in range(scl_th):
                    for px in range(scl_th):
                        pxl[py][px] = sgmnt_img[y+py-scl_th+1][x+px-scl_th+1]/255.0
                
                pxl_x_diff  = np.zeros((scl_th, 3))
                for py in range(scl_th):
                    for px in range(1, scl_th):
                        pxl_x_diff[py] += np.abs(pxl[py][px-1] - pxl[py][px])

                pxl_x = 0.0
                for seq in range(3):
                    for py in range(scl_th):
                        if pxl_x_diff[py][seq] > clr_th:
                            pxl_x += 1.0
                

                pxl_y_diff  = np.zeros((scl_th, 3))
                for py in range(1, scl_th):
                    for px in range(scl_th):
                        pxl_y_diff[px] += np.abs(pxl[py-1][px] - pxl[py][px])
                                
                pxl_y = 0.0
                for seq in range(3):
                    for py in range(scl_th):
                         if pxl_y_diff[py][seq] > clr_th:
                            pxl_y += 1.0

                
                if (pxl_y + pxl_x) > 1/(scl_th*scl_th*255.0):
                    edges_img[y][x][0] = 1.0
                    edges_img[y][x][1] = 1.0
                    edges_img[y][x][2] = 1.0
        
        
        edges_img = cv2.threshold(edges_img, 0.5, 1.0, cv2.THRESH_BINARY)[1]
        plt.imsave(panopic_edge_img_kitti+"/"+file_name,  edges_img)
        #print(np.shape(edges_img))
        #plt.imshow(edges_img)
        #plt.show()

print('Edge Extraction is finished.')

### Check Images

In [ ]:
img_count = 0
for file_name in files:
    if '.jpg' in file_name:
        img_count += 1

        file_path = path_depth+file_name

        orign_img = cv2.imread(file_path)
        orign_img = np.asarray(orign_img, dtype=np.float32)
        orign_img = cv2.cvtColor(orign_img, cv2.COLOR_BGR2RGB)
        orign_img = orign_img[:128, :416]/255.0
        y, x, s   = np.shape(orign_img)
        #print(np.shape(orign_img))
        #plt.imshow(orign_img)
        #plt.show()
        
        file_path = panopic_img_kitti+"/"+file_name[0:10]+".jpg"
        sgmnt_img = cv2.imread(file_path)
        sgmnt_img = np.asarray(sgmnt_img, dtype=np.float32)
        sgmnt_img = cv2.cvtColor(sgmnt_img, cv2.COLOR_BGR2RGB)
        sgmnt_img = cv2.resize(sgmnt_img, (x, y))/255.0
        #print(np.shape(sgmnt_img))
        #plt.imshow(sgmnt_img)
        #plt.show()        
        
        file_path = "./depth_img/"+file_name[0:10]+'.png'
        depth_img = cv2.imread(file_path)
        depth_img = np.asarray(depth_img, dtype=np.float32)
        depth_img = cv2.cvtColor(depth_img, cv2.COLOR_BGR2RGB)
        depth_img = cv2.resize(depth_img, (x, y))/255.0
        #print(np.shape(depth_img))
        #plt.imshow(depth_img)
        #plt.show() 

        file_path = panopic_edge_img_kitti+"/"+file_name
        edges_img = cv2.imread(file_path)
        edges_img = np.asarray(edges_img, dtype=np.float32)
        edges_img = cv2.cvtColor(edges_img, cv2.COLOR_BGR2RGB)
        edges_img = cv2.resize(edges_img, (x, y))/255.0
        #print(np.shape(edges_img))
        #plt.imshow(edges_img)
        #plt.show()    
        
        img0 = cv2.hconcat([orign_img, sgmnt_img])
        img1 = cv2.hconcat([depth_img, edges_img])
        img2 = cv2.vconcat([img0, img1])
        
        depsed_segment_img = depth_img * edges_img
        img3 = cv2.resize(depsed_segment_img, (x*2, y*2))
        img  = cv2.vconcat([img2, img3])
        #plt.imshow(img)
        #plt.show()

        file_path = chk_image_kitti+"/"+file_name
        plt.imsave(file_path,  img)
        
        file_path = seg_depth_img_kitti+"/"+file_name
        plt.imsave(file_path,  depsed_segment_img)

print('Image Check Finished.')

### Point Cloud Constructor

In [ ]:
def depthToPoint3d(depth):
    xx, yy = np.meshgrid(np.arange(0, width), np.arange(0, height))
    x = xx
    y = yy
    depth = 4095 * depth
    pos3d = np.dstack((x, y, depth, depth)).transpose(2, 1, 0).reshape(4, width*height)
    return pos3d[0:3,:]

def generatePointCloud(depth):
    point3d = depthToPoint3d(depth).T/10000.0
    points = pd.DataFrame(point3d)
    points.columns = ['x', 'y', 'z']
    cloud = PyntCloud(points)
    return cloud

def axisPointCloud(scale):
    # color, [vertexes]
    lines = {
        '0xFF0000': [[0, 0, 0], [scale, 0, 0]],
        '0x00FF00': [[0, 0, 0], [0, scale, 0]],
        '0x0000FF': [[0, 0, 0], [0, 0, scale]]
    }
    return lines

### Point-Clout Display

In [ ]:
img_count = 0
for file_name in files:
    if '.jpg' in file_name:
        img_count += 1
        print('{}: {}'.format(img_count, file_name))
        
        file_path = "./seg_depth_img_kitti/"+file_name
        
        seg_depth_img = cv2.imread(file_path)
        seg_depth_img = np.asarray(seg_depth_img, dtype=np.float32)
        seg_depth_img = cv2.cvtColor(seg_depth_img, cv2.COLOR_BGR2RGB)
        seg_depth_img = seg_depth_img/255.0

        cloud = generatePointCloud(seg_depth_img[:,:,0])
        #cloud.plot(mesh=True, backend="threejs")
        #cloud.plot(point_size=0.0001, opacity=1.0, polylines=axisPointCloud(1.0), elev=145, azim=110)
        #cloud.to_file("./semantic_point_cloud_kitti/"+file_name[0:10]+".dat", ext='CSV')        